In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate() 

from pydataset import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark


## Data Acquisition

These exercises should go in a notebook or script named 'wrangle'. Add, commit, and push your changes.

This exercises uses the 'case.csv', 'dept.csv', and 'source.csv' files from the san antonio 311 call dataset.

<font color="blue">__1. Read the case, department, and source data into their own spark dataframes.__</font>



In [2]:
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)
case = spark.read.csv('case.csv', sep=',', header=True, inferSchema=True)
dept = spark.read.csv('dept.csv', sep=',', header=True, inferSchema=True)

<font color="blue">__2. Let's see how writing to the local disk works in spark:__</font>

   * Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
   * Inspect your folder structure. What do you notice?


In [3]:
source.write.csv('sources_csv', mode='overwrite')
source.write.json('sources_json', mode='overwrite')

<font color="blue">__3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.__



In [4]:
source.printSchema(), case.printSchema(), dept.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



(None, None, None)

In [5]:
source.show(), source.printSchema()

+---------+--------------------+
|source_id|     source_username|
+---------+--------------------+
|   100137|    Merlene Blodgett|
|   103582|         Carmen Cura|
|   106463|     Richard Sanchez|
|   119403|      Betty De Hoyos|
|   119555|      Socorro Quiara|
|   119868| Michelle San Miguel|
|   120752|      Eva T. Kleiber|
|   124405|           Lori Lara|
|   132408|       Leonard Silva|
|   135723|        Amy Cardenas|
|   136202|    Michelle Urrutia|
|   136979|      Leticia Garcia|
|   137943|    Pamela K. Baccus|
|   138605|        Marisa Ozuna|
|   138650|      Kimberly Green|
|   138650|Kimberly Green-Woods|
|   138793| Guadalupe Rodriguez|
|   138810|       Tawona Martin|
|   139342|     Jessica Mendoza|
|   139344|        Isis Mendoza|
+---------+--------------------+
only showing top 20 rows

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



(None, None)

In [6]:
case.show(), case.printSchema()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

(None, None)

In [7]:
dept.show(), dept.printSchema()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

(None, None)

1. __How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?__


2. __How many Stray Animal cases are there?__



In [8]:
case.select('service_request_type').groupBy('service_request_type').count().where(case.service_request_type == 'Stray Animal').show(truncate=False)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|Stray Animal        |26760|
+--------------------+-----+



3. __How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?__



In [9]:
case.select('service_request_type').where(case.dept_division == 'Field Operations').where(case.service_request_type != 'Officer Standby').count()

113902

4. __Convert the `council_district` column to a string column.__



In [10]:
case.select('service_request_type').where(case.dept_division == 'Field Operations').where(case.service_request_type != 'Officer Standby').count()

113902

5. __Extract the year from the `case_closed_date` column.__



6. __Convert `num_days_late` from days to hours in new columns `num_hours_late.`



7. __Join the case data with the source and department data.__



8. __Are there any cases that do not have a request source?



9. __What are the top 10 service request types in terms of number of requests?__



10. __What are the top 10 service request types in terms of average days late?__



12. __How do number of days late depend on department and request type?__